In [ ]:
import tensorflow as tf
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense
from keras.models import Sequential
from gensim.models import FastText  # Adjust if using a different FastText library
import fasttext.util
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.regularizers import l1



# Load and preprocess text data
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2) # Replace with your data splitting function

# Tokenize text (if not already pre-tokenized)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to a uniform length
max_length = 100  # Adjust as needed
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_length)

# Load or create FastText model
fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")
vocab_size = len(tokenizer.word_index) + 1  # Account for unknown words
embedding_dim = 300  # Match FastText embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Define CNN-BiLSTM model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test accuracy:", accuracy)


In [4]:
model.save('D:\Mini Project\minipj\models\CNN_BiLSTM_model1.h5')  # Replace with your desired path and filename

C:\Users\rosha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import tensorflow as tf
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense
from keras.models import Sequential
from gensim.models import FastText  # Adjust if using a different FastText library
import fasttext.util
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.regularizers import l1



# Load and preprocess text data
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2) # Replace with your data splitting function

# Tokenize text (if not already pre-tokenized)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to a uniform length
max_length = 100  # Adjust as needed
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_length)

# Load or create FastText model
fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")
vocab_size = len(tokenizer.word_index) + 1  # Account for unknown words
embedding_dim = 300  # Match FastText embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l1(0.01)),  # L1 regularization added
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(32, dropout=0.2)),  # Dropout added
    Dense(16, activation='relu', kernel_regularizer=l1(0.01)),  # L1 regularization added
    Dense(1, activation='sigmoid')  # Output layer
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Adjusted loss for binary classification

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test accuracy:", accuracy)


Epoch 1/10
5802/5802 [==============================] - 213s 36ms/step - loss: 0.7757 - accuracy: 0.8116 - val_loss: 0.5859 - val_accuracy: 0.8430
Epoch 2/10
5802/5802 [==============================] - 112s 19ms/step - loss: 0.5851 - accuracy: 0.8484 - val_loss: 0.5454 - val_accuracy: 0.8663
Epoch 3/10
5802/5802 [==============================] - 115s 20ms/step - loss: 0.5759 - accuracy: 0.8498 - val_loss: 0.5848 - val_accuracy: 0.8488
Epoch 4/10
5802/5802 [==============================] - 113s 20ms/step - loss: 0.5668 - accuracy: 0.8530 - val_loss: 0.7909 - val_accuracy: 0.7426
Epoch 5/10
5802/5802 [==============================] - 117s 20ms/step - loss: 0.5639 - accuracy: 0.8543 - val_loss: 0.5705 - val_accuracy: 0.8633
Epoch 6/10
5802/5802 [==============================] - 126s 22ms/step - loss: 0.5616 - accuracy: 0.8558 - val_loss: 0.5125 - val_accuracy: 0.8826
Epoch 7/10
5802/5802 [==============================] - 136s 23ms/step - loss: 0.5519 - accuracy: 0.8585 - val_loss: 0

In [5]:
model.save('D:\Mini Project\minipj\models\CNN_BiLSTM_model2.h5')

C:\Users\rosha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


: 

In [1]:
import tensorflow as tf
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense
from keras.models import Sequential
from gensim.models import FastText  # Adjust if using a different FastText library
import fasttext.util
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.regularizers import l2



# Load and preprocess text data
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2) # Replace with your data splitting function

# Tokenize text (if not already pre-tokenized)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to a uniform length
max_length = 100  # Adjust as needed
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_length)

# Load or create FastText model
fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")
vocab_size = len(tokenizer.word_index) + 1  # Account for unknown words
embedding_dim = 300  # Match FastText embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(32, dropout=0.2)),  # Dropout
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
    Dense(1, activation='sigmoid')  # Output layer
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Adjusted loss for binary classification

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test accuracy:", accuracy)






Epoch 1/10


5802/5802 [==============================] - 124s 20ms/step - loss: 0.3751 - accuracy: 0.8750 - val_loss: 0.3104 - val_accuracy: 0.9015
Epoch 2/10
5802/5802 [==============================] - 119s 21ms/step - loss: 0.3201 - accuracy: 0.8923 - val_loss: 0.2912 - val_accuracy: 0.9014
Epoch 3/10
2849/5802 [=============>................] - ETA: 53s - loss: 0.3068 - accuracy: 0.8963

KeyboardInterrupt: 

In [4]:
import tensorflow as tf
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense
from keras.models import Sequential
from gensim.models import FastText  # Adjust if using a different FastText library
import fasttext.util
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.regularizers import l2



# Load and preprocess text data
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2) # Replace with your data splitting function

# Tokenize text (if not already pre-tokenized)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to a uniform length
max_length = 100  # Adjust as needed
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_length)

# Load or create FastText model
fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")
vocab_size = len(tokenizer.word_index) + 1  # Account for unknown words
embedding_dim = 300  # Match FastText embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [1]:
import tensorflow as tf
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

loaded_model = load_model("D:\Mini Project\minipj\models\CNN_BiLSTM_model2.h5")

loss, accuracy = loaded_model.evaluate(X_test_padded, y_test)
print("Test accuracy:", accuracy)

KeyboardInterrupt: 

: 